In [ ]:
import json
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt

with open('data/commonFriends.json', 'r') as file:
    friends_links = json.load(file)

with open('data/myFriends.json', 'r') as file:
    friends_info = json.load(file)
    
friends_info = friends_info['response']['items']

In [ ]:
# graph:
sex = dict(map(lambda d: (d['id'], -d['sex']), friends_info))

G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    G.add_edges_from([(int(friend_ID), common_friend_ID) 
                      for common_friend_ID in common_friend_IDs])
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

# sex

In [ ]:
low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)
    
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.2)
nx.draw(G, pos, nodelist=sex,
        node_color=[mapper.to_rgba(i) 
                    for i in sex.values()],
        width=2, with_labels=False, node_size = 100)
plt.savefig("graphs/friends_sex.png")
plt.show()

# city

In [ ]:
city = dict(map(lambda d: (d['id'], d['city']['id']) 
                if 'city' in d.keys() else (d['id'], -1), friends_info))

In [ ]:
cities_id_map = np.unique(list(city.values()))
cities_id_map = cities_id_map[cities_id_map!=-1]
cities_id_map = {k: v for v, k in enumerate(cities_id_map)}

users_with_city = {k:v for k, v in city.items() if v!=-1}

low, *_, high = cities_id_map
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_city = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.rainbow)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    for edge in edges_list:
        if (city[edge[0]]==city[edge[1]])&(city[edge[0]]!=-1):
            color_id = city[edge[0]]
            G.add_edges_from([edge], 
                             color = mapper_city.to_rgba(cities_id_map[color_id]))
        #else:
        #    G.add_edges_from([edge], color = 'white')
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]

    
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.2)

nx.draw(G, pos, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()],
        edges=edges, edge_color=colors, 
        width=2, with_labels=False, node_size = 100)

plt.savefig("graphs/friends_sex.png")
plt.show()

# personal

In [ ]:
personal = dict(map(lambda d: (d['id'],d['personal']) 
                    if 'personal' in d.keys() else (-1, -1), friends_info))
if -1 in personal:
    del personal[-1]

In [ ]:
political = dict([(k,v['political']) for k, v in personal.items() if 'political' in v.keys()])
religion = dict([(k,v['religion']) for k, v in personal.items() if 'religion' in v.keys()])

In [ ]:
x, height = np.unique(list(political.values()), return_counts=True)
plt.figsize = (20,15)
fig, ax = plt.subplots()

rects = ax.bar(x, height)

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 0.5),
                    textcoords="offset points",
                    ha='center', va='bottom')
autolabel(rects)


ax.set_ylabel('Amount')
ax.set_title('My friends\' political views')

ax.legend()

fig.tight_layout()
plt.savefig("graphs/friends_political_bar.png")
plt.show()

In [ ]:
print('Total filling: ', np.sum(height))

In [ ]:
low, *_, high = sorted(list(political.values()))
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_political = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.spring)

In [ ]:
G = nx.Graph()
for friend_ID, common_friend_IDs in friends_links.items():
    edges_list = [(int(friend_ID), common_friend_ID) 
                  for common_friend_ID in common_friend_IDs]
    if int(friend_ID) in political:
        for edge in edges_list:
            if (edge[1] in political):
                if (political[edge[0]]==political[edge[1]])&(political[edge[0]]!=-1):
                    color_id = political[edge[0]]
                    G.add_edges_from([edge], 
                                     color = mapper_political.to_rgba(color_id))
            else:
                G.add_edges_from([edge], color = 'white')
            
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

edges = G.edges()
colors = [G[u][v]['color'] for u,v in edges]

plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.2)
nx.draw(G, pos, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()],
        edges=edges, edge_color=colors,
        width=2, with_labels=False, node_size = 100)
plt.savefig("graphs/friends_political.png")
plt.show()